# Indexing script for Compact Memory fulltext search

## Imports

In [1]:
import os, json, pprint, time, sys, requests, pysolr, subprocess, json, pprint
base = '/data/cm/output/'
SOLR_INDEX = "cm_meta"

## Read Journals

In [2]:
c = 0
chunk = 0
data = []
CHUNK_TARGET = 100000  # grober Schwellenwert in Bytes

input_root = os.path.join(base, "solr_meta_fra")
output_root = os.path.join(base, "output/solr_temp_fra")

os.makedirs(output_root, exist_ok=True)

for journal in os.listdir(input_root):
    journal_dir = os.path.join(input_root, journal)
    if not os.path.isdir(journal_dir):
        continue

    print("Processing", journal, "Chunk", str(chunk), end=" ")

    for file in os.listdir(journal_dir):
        file_path = os.path.join(journal_dir, file)
        if not os.path.isfile(file_path):
            continue

        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        if len(lines) < 2:
            # unerwartetes Format, überspringen
            continue

        header = json.loads(lines[0].strip())
        body = json.loads(lines[1].strip())

        # ID aus Header (_id) oder Body übernehmen → Solr-Feld "id"
        doc_id = header.get("id") or header.get("_id") or body.get("id")
        if doc_id is None:
            # Wenn keine ID vorhanden ist, notfalls aus Dateiname bauen
            doc_id = os.path.splitext(file)[0]
        body["id"] = doc_id

        # Nur erlaubte Felder übernehmen
        allowed_fields = {
            "id", "text", "dateIssued", "lang", "place", "j_title", "publisher",
            "volume", "heft", "aufsatz", "page", "vlid_journal", "vlid_page"
        }
        body = {k: v for k, v in body.items() if k in allowed_fields}

        # Solr-JSONL: ein Dokument pro Zeile
        data.append(json.dumps(body, ensure_ascii=False))
        c += 1

        # Grob nach Größe chunken
        if sys.getsizeof(data) > CHUNK_TARGET:
            out_path = os.path.join(output_root, f"chunk_{chunk}.jsonl")
            with open(out_path, "w", encoding="utf-8") as write_file:
                write_file.write("\n".join(data))
            print(f"\n  -> wrote {len(data)} docs to {out_path}")
            chunk += 1
            data = []

# letzten Chunk schreiben
if data:
    out_path = os.path.join(output_root, f"chunk_{chunk}.jsonl")
    with open(out_path, "w", encoding="utf-8") as write_file:
        write_file.write("\n".join(data))
    print(f"\n  -> wrote final {len(data)} docs to {out_path}")

print(f"\nFertig. Insgesamt {c} Dokumente verarbeitet.")

Processing 2710055 Chunk 0 Processing 9616701 Chunk 0 Processing 2580773 Chunk 0 Processing 2895450 Chunk 0 Processing 2727810 Chunk 0 Processing 3129675 Chunk 0 Processing 6492429 Chunk 0 Processing 2438141 Chunk 0 Processing 2827798 Chunk 0 Processing 5438908 Chunk 0 Processing 4782723 Chunk 0 Processing 7506414 Chunk 0 Processing 9054153 Chunk 0 Processing 9366335 Chunk 0 Processing 4889892 Chunk 0 Processing 2425321 Chunk 0 Processing 6492329 Chunk 0 Processing 700673 Chunk 0 Processing 4895413 Chunk 0 Processing 2542917 Chunk 0 Processing 9583346 Chunk 0 Processing 2549643 Chunk 0 Processing 10638461 Chunk 0 Processing 3139318 Chunk 0 Processing 9620169 Chunk 0 Processing 9051331 Chunk 0 Processing 4781937 Chunk 0 Processing 4911569 Chunk 0 Processing 2625941 Chunk 0 Processing 2446951 Chunk 0 Processing README.md Chunk 0 Processing 9643693 Chunk 0 Processing 2912821 Chunk 0 Processing 6465881 Chunk 0 Processing 9643701 Chunk 0 Processing 3020846 Chunk 0 Processing 9230930 Chunk 0

In [3]:
c = 0
chunk = 0
data = []

for journal in os.listdir(base+'solr_meta_fra/'):
    
    print('Processing', journal, 'Chunk', str(chunk), end=' ')
    
    
    for file in os.listdir(base+'solr_meta_fra/'+journal+'/'):
        
        f = open(base+'solr_meta_fra/'+journal+'/'+file, 'r', encoding='utf-8').readlines()
                
        #header = json.loads(f[0].strip('\n'))
        body = json.loads(f[0].strip('\n'))
        
        # Optional: nur erlaubte Felder übernehmen
        allowed_fields = {
            "id", "text", "dateIssued", "lang", "place", "j_title", "publisher",
            "volume", "heft", "aufsatz", "page", "vlid_journal", "vlid_page"
        }
        body = {k: v for k, v in body.items() if k in allowed_fields}
        
        # Nur Body (das eigentliche Dokument) speichern
        data.append(json.dumps(body, ensure_ascii=False))

        if sys.getsizeof(data) > 100000:
            with open('/data/cm/output/solr_temp_fra/chunk_'+str(chunk)+'.json', "w", encoding="utf-8") as write_file:
                #data = [json.dumps(d, ensure_ascii=False) for d in data]
                write_file.write("\n".join(data))
                
            chunk += 1
            data = []


Processing 2710055 Chunk 0 Processing 9616701 Chunk 0 Processing 2580773 Chunk 0 Processing 2895450 Chunk 0 Processing 2727810 Chunk 1 Processing 3129675 Chunk 1 Processing 6492429 Chunk 1 Processing 2438141 Chunk 1 Processing 2827798 Chunk 2 Processing 5438908 Chunk 2 Processing 4782723 Chunk 2 Processing 7506414 Chunk 2 Processing 9054153 Chunk 2 Processing 9366335 Chunk 2 Processing 4889892 Chunk 2 Processing 2425321 Chunk 2 Processing 6492329 Chunk 2 Processing 700673 Chunk 2 Processing 4895413 Chunk 2 Processing 2542917 Chunk 2 Processing 9583346 Chunk 3 Processing 2549643 Chunk 3 Processing 10638461 Chunk 3 Processing 3139318 Chunk 3 Processing 9620169 Chunk 3 Processing 9051331 Chunk 3 Processing 4781937 Chunk 3 Processing 4911569 Chunk 3 Processing 2625941 Chunk 3 Processing 2446951 Chunk 3 Processing README.md Chunk 10 Processing 9643693 Chunk 10 Processing 2912821 Chunk 10 Processing 6465881 Chunk 10 Processing 9643701 Chunk 10 Processing 3020846 Chunk 10 Processing 9230930 C

## Create SOLR Index

In [4]:
def index_exists(solr_url):
    try:
        solr = pysolr.Solr(solr_url, timeout=5)
        solr.ping()  # sendet eine einfache Anfrage
        return True
    except Exception as e:
        print(f"Fehler: {e}")
        return False

url = f"http://localhost:8983/solr/{SOLR_INDEX}"
if index_exists(url):
    print("Index exists. Removing...")
    url = f"http://localhost:8983/solr/admin/cores?action=UNLOAD&core={SOLR_INDEX}&deleteInstanceDir=true"
    response = requests.get(url)
    print(response.status_code)
    print(response.text)
else: 
    print("Index does not exist. Creating ...")


url = f"http://localhost:8983/solr/admin/cores?action=CREATE&name={SOLR_INDEX}&instanceDir={SOLR_INDEX}&configSet=_default"

response = requests.get(url)
print(response.status_code)
print(response.text)

Fehler: Solr responded with an error (HTTP 404): [Reason: None]
<p>  Searching for Solr?  You must type the correct path.  Solr will respond.</p>
Index does not exist. Creating ...
200
{
  "responseHeader":{
    "status":0,
    "QTime":950
  },
  "core":"cm_meta"
}


## Test the CORE

In [5]:
solr = pysolr.Solr(f'http://localhost:8983/solr/{SOLR_INDEX}/', always_commit=True, timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":1,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-18"\n    }\n  },\n  "status":"OK"\n}'

## Add fields to the core

In [6]:
url = f"http://localhost:8983/solr/{SOLR_INDEX}/schema"

payload = {
    "add-field": [
        { "name": "id",         "type": "text_general", "stored": True, "indexed": True },
        { "name": "text",         "type": "text_general", "stored": True, "indexed": True },
        { "name": "dateIssued",   "type": "plong", "stored": True, "indexed": True },
        { "name": "lang",         "type": "text_general", "stored": True, "indexed": True },
        { "name": "place",        "type": "text_general", "stored": True, "indexed": True },
        { "name": "j_title",      "type": "text_general", "stored": True, "indexed": True },
        { "name": "publisher",    "type": "text_general", "stored": True, "indexed": True },
        { "name": "volume",       "type": "text_general", "stored": True, "indexed": True },
        { "name": "heft",         "type": "text_general", "stored": True, "indexed": True },
        { "name": "aufsatz",      "type": "text_general", "stored": True, "indexed": True },
        { "name": "page",         "type": "text_general", "stored": True, "indexed": True },
        { "name": "vlid_journal", "type": "plong", "stored": True, "indexed": True },
        { "name": "vlid_page",    "type": "plong", "stored": True, "indexed": True }
    ]
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, data=json.dumps(payload))

print(f"Status Code: {response.status_code}")
print(response.text)

Status Code: 400
{
  "responseHeader":{
    "status":400,
    "QTime":29
  },
  "error":{
    "metadata":["error-class","org.apache.solr.api.ApiBag$ExceptionWithErrObject","root-error-class","org.apache.solr.api.ApiBag$ExceptionWithErrObject"],
    "details":[{
      "add-field":{
        "name":"id",
        "type":"text_general",
        "stored":true,
        "indexed":true
      },
      "errorMessages":["Field 'id' already exists.\n"]
    },{
      "add-field":{
        "name":"text",
        "type":"text_general",
        "stored":true,
        "indexed":true
      },
      "errorMessages":["Field 'text' already exists.\n"]
    },{
      "add-field":{
        "name":"dateIssued",
        "type":"plong",
        "stored":true,
        "indexed":true
      },
      "errorMessages":["Field 'dateIssued' already exists.\n"]
    },{
      "add-field":{
        "name":"lang",
        "type":"text_general",
        "stored":true,
        "indexed":true
      },
      "errorMessages":["Fi

In [7]:
## Test the fields

In [8]:
url = f"http://localhost:8983/solr/{SOLR_INDEX}/schema/fields?wt=json"
response = requests.get(url)

print(f"Status Code: {response.status_code}")
print(response.text)

Status Code: 200
{
  "responseHeader":{
    "status":0,
    "QTime":15
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"aufsatz",
    "type":"text_general",
    "indexed":true,
    "stored":true
  },{
    "name":"date",
    "type":"text_general"
  },{
    "name":"dateIssued",
    "type":"text_general",
    "indexed":true,
    "stored":true
  },{
    "name":"e_type",
    "type":"text_general"
  },{
    "name":"end",
    "type":"plongs"
  },{
    "name":"ep",
    "type":"text_general"
  },{
    "name":"first",
    "type":"plongs"
  },{
    "name":"heft",
    "type":"text_general",
    "indexed":true,
    "stored":true
  },{
    "name":"id

## Upload all the JSON Files 

In [9]:
allowed_fields = {
    "id", "text", "dateIssued", "lang", "place", "j_title", "publisher",
    "volume", "heft", "aufsatz", "page", "vlid_journal", "vlid_page"
}


In [10]:
solr_core_url = f"http://localhost:8983/solr/{SOLR_INDEX}/update"

In [11]:
start = time.time()

for file in sorted(os.listdir(input_root)):
    path = os.path.join(input_root, file)
    if not os.path.isfile(path):
        continue

    print("Processing:", file)

    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    docs = []

    for line in lines:
        line = line.strip()
        if not line:
            continue

        try:
            raw = json.loads(line)
        except json.JSONDecodeError as e:
            print(f"⚠️ JSON-Fehler in {file}: {e}")
            continue

        # Nur erlaubte Felder übernehmen
        filtered = {k: v for k, v in raw.items() if k in allowed_fields}

        # Solr braucht ein 'id'-Feld – ohne ID nicht indexieren
        if "id" not in filtered or not filtered["id"]:
            print(f"⚠️ Keine 'id' im Dokument (Datei {file}), überspringe Eintrag.")
            continue

        docs.append(filtered)

    if not docs:
        print(f"⚠️ Keine gültigen Dokumente in {file}, übersprungen.")
        continue

    # an Solr senden – Liste von Dokumenten ist für die JSON Update API gültig
    try:
        response = requests.post(
            solr_core_url,
            headers={"Content-Type": "application/json"},
            data=json.dumps(docs, ensure_ascii=False),
            params={"commit": "true"},
            timeout=200,
        )
    except requests.RequestException as e:
        print(f"❌ Request-Fehler bei {file}: {e}")
        continue

    print(f"{file} → Status {response.status_code}")
    if response.status_code != 200:
        print("Antwort von Solr:")
        print(response.text)

end = time.time()
print("Indexing time:", str(end - start))

Indexing time: 0.006428241729736328


In [12]:
solr_core_url = f"http://localhost:8983/solr/{SOLR_INDEX}/update"


start = time.time()

for file in os.listdir('/data/cm/output/solr_temp_fra/'):
    print("Processing:", file)

    path = os.path.join('/data/cm/output/solr_temp_fra/', file)

    if not os.path.isfile(path):
        continue 
        
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    docs = []

    for line in lines:
        if not line.strip():
            continue
        try:
            raw = json.loads(line)
        except json.JSONDecodeError as e:
            print(f"⚠️ JSON-Fehler: {e}")
            continue
    
    
        # Nur erlaubte Felder übernehmen
        filtered = {k: v for k, v in raw.items() if k in allowed_fields}
        #if "id" in filtered:  # nur hinzufügen, wenn ID gesetzt
        docs.append(filtered)
        #print(filtered)

    # an Solr senden
    response = requests.post(
        solr_core_url,
        headers={"Content-Type": "application/json"},
        data=json.dumps(docs),
        params={"commit": "true"},
        timeout=200
    )

    print(f"{file} → Status {response.status_code}")
    if response.status_code != 200:
        print(response.text)

end = time.time()
print("Indexing time:", str(end - start))

Processing: chunk_30.json
chunk_30.json → Status 200
Processing: chunk_50.json
chunk_50.json → Status 200
Processing: chunk_12.json
chunk_12.json → Status 200
Processing: chunk_40.json
chunk_40.json → Status 200
Processing: chunk_2.json
chunk_2.json → Status 200
Processing: chunk_43.json
chunk_43.json → Status 200
Processing: chunk_32.json
chunk_32.json → Status 200
Processing: chunk_28.json
chunk_28.json → Status 200
Processing: chunk_58.json
chunk_58.json → Status 200
Processing: chunk_22.json
chunk_22.json → Status 200
Processing: chunk_56.json
chunk_56.json → Status 200
Processing: chunk_42.json
chunk_42.json → Status 200
Processing: chunk_46.json
chunk_46.json → Status 200
Processing: chunk_39.json
chunk_39.json → Status 200
Processing: chunk_14.json
chunk_14.json → Status 200
Processing: chunk_60.json
chunk_60.json → Status 200
Processing: chunk_18.json
chunk_18.json → Status 200
Processing: chunk_49.json
chunk_49.json → Status 200
Processing: chunk_33.json
chunk_33.json → Status

## Test query

In [13]:
core_name = "cm_meta"
query = "text:Turin"
url = f"http://localhost:8983/solr/{core_name}/select"

params = {
    "q": query,
    "wt": "json"
}

start = time.time()
response = requests.get(url, params=params)
end = time.time()

print("Search took", round(end - start, 2), "seconds")

if response.status_code == 200:
    res = response.json()
    results = response.json()["response"]["docs"]
    print("Found", len(results), "documents")
    for doc in results:
        print(doc)
else:
    print("Error:", response.status_code)
    print(response.text)


Search took 0.03 seconds
Found 10 documents
{'id': '07719_2947283', 'text': ["596 Die Rabbknen in Venedig und Turin wohnen unweit oder in den ehemaligen Ghettis, die Gottlob nunmehr gefallen sind. — Der Rabbiner in Venedig leitet die Talmud-Thora und ist Borsteher der Schomcrlaboker Chevra, die Morgens um sechs Uhr Gottesdienst hält, und wozu er auch regelmäßig erscheint. In Verona hat seit einigen Jahren ein jüdischer Bankier eine Mtkwoh unweit seines Hauses mit der Gemeinde gebaut. Mailand zählt ungefähr 1000 jüdischer Seelen, die, obgleich sie eine Synagoge haben, tm übrigen doch ohne weiteren Zusammenhang leben. Die jüdischen Zustände in Turin haben das wohlthuende Aeußere einer frommen jüdischen Gemeinde; man zählt daselbst eine italienische und eine spanische und eine Synagoge des deutschen MinHags; die Aussprache in allen Synagogen Italiens ist die sephardische; man baut in Turin eine neue Synagoge. Der Samstagsgottesdienst beginnt um acht Uhr; die Synagogen sind ziemlich voll; 

In [14]:
print(res.keys())  # z. B. dict_keys(['responseHeader', 'response'])
print(len(res['response']['docs']))  # Anzahl gefundener Dokumenteprint(res.keys())  # z. B. dict_keys(['responseHeader', 'response'])
print(len(res['response']['docs']))  # Anzahl gefundener Dokumente

dict_keys(['responseHeader', 'response'])
10
10


In [15]:

print("Total hits:", res["response"]["numFound"])

# Solr-Treffer sind in res['response']['docs']
for doc in res["response"]["docs"]:
    print("Id:", doc.get("id", "(kein ID-Feld)"))
    print("Score:", doc.get("score"))
    print("Text:", doc.get("text", "")[:100])
    print("Date Issued:", doc.get("dateIssued"))
    print("Language:", doc.get("lang"))
    print("Place:", doc.get("place"))
    print("Journal Title:", doc.get("j_title"))
    print("Publisher:", doc.get("publisher"))
    print("Volume:", doc.get("volume"))
    print("Heft:", doc.get("heft"))
    print("Aufsatz:", doc.get("aufsatz"))
    print("Page:", doc.get("page"))
    print("VLID_journal:", doc.get("vlid_journal"))
    print("VLID_page:", doc.get("vlid_page"))
    print("-" * 40)
    break  # Nur ersten Treffer anzeigen

Total hits: 1592
Id: 07719_2947283
Score: None
Text: ["596 Die Rabbknen in Venedig und Turin wohnen unweit oder in den ehemaligen Ghettis, die Gottlob nunmehr gefallen sind. — Der Rabbiner in Venedig leitet die Talmud-Thora und ist Borsteher der Schomcrlaboker Chevra, die Morgens um sechs Uhr Gottesdienst hält, und wozu er auch regelmäßig erscheint. In Verona hat seit einigen Jahren ein jüdischer Bankier eine Mtkwoh unweit seines Hauses mit der Gemeinde gebaut. Mailand zählt ungefähr 1000 jüdischer Seelen, die, obgleich sie eine Synagoge haben, tm übrigen doch ohne weiteren Zusammenhang leben. Die jüdischen Zustände in Turin haben das wohlthuende Aeußere einer frommen jüdischen Gemeinde; man zählt daselbst eine italienische und eine spanische und eine Synagoge des deutschen MinHags; die Aussprache in allen Synagogen Italiens ist die sephardische; man baut in Turin eine neue Synagoge. Der Samstagsgottesdienst beginnt um acht Uhr; die Synagogen sind ziemlich voll; der Rabbiner hielt nach